# 네이버 매장 -> 매장에 있는 tag 

문제
 
1. 추천순으로 긁어올지, 최신순으로 긁어올지

In [35]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException

from bs4 import BeautifulSoup

import datetime
import time
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import requests
from openpyxl import Workbook
from bs4 import BeautifulSoup
import pandas as pd


In [5]:
# 데이터 불러오기
original_res_df = pd.read_excel('combined_file.xlsx')
res_df = original_res_df.copy()
res_df = res_df.loc[res_df['검색여부'] == '가능'].reset_index(drop=True)
res_df.drop('Unnamed: 0', axis=1, inplace=True)

res_df.tail()

,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,사업장명_원본,검색여부
8291,영업/정상,서울특별시 강남구 대치동 937 롯데백화점,"서울특별시 강남구 도곡로 401, 롯데백화점 지하1층 (대치동)",간코,일식,204669.543367,443873.621189,서울특별시,강남구,"도곡로 401,",간코 롯데백화점강남점,가능
8292,영업/정상,서울특별시 강남구 신사동 528-3 덕산빌딩,"서울특별시 강남구 압구정로 108, 덕산빌딩 지상1층 10호 (신사동)",오토김밥,분식,201749.456414,446742.828124,서울특별시,강남구,"압구정로 108,",오토김밥 신사점,가능
8293,영업/정상,서울특별시 강남구 신사동 635-14 금광아크존1,"서울특별시 강남구 압구정로42길 36, 지상1층 101호 (신사동, 금광아크존1)",마들마들,한식,202990.870783,447146.555568,서울특별시,강남구,"압구정로42길 36,",마들마들,가능
8294,영업/정상,서울특별시 강남구 신사동 629-1,"서울특별시 강남구 언주로151길 21, 지상1층 (신사동)",세바,경양식,202731.421127,446609.460134,서울특별시,강남구,"언주로151길 21,",세바(Sevva),가능
8295,영업/정상,서울특별시 강남구 신사동 552-5,"서울특별시 강남구 논현로161길 37, 동쪽 지상1층 (신사동)",구구구,경양식,202153.410454,446748.562578,서울특별시,강남구,"논현로161길 37,",구구구 (GOOGOOGOO),가능


In [9]:
# 해민
# driver = webdriver.Chrome()

In [64]:
# test 
res_df = res_df[:3]
res_df

,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,사업장명_원본,검색여부
0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,한식,202203.971845,444853.761458,서울특별시,강남구,"강남대로114길 18,",정식당,가능
1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트,한식,202462.794327,443827.961945,서울특별시,강남구,"강남대로 378,",채선당 샤브보트 강남역점,가능
2,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,한식,203470.848439,447369.579852,서울특별시,강남구,"압구정로 343,",수아당,가능


In [66]:
# 지선
driver = webdriver.Chrome(ChromeDriverManager().install())

# 빈 리스트 생성
restaurant_name_list = []
category_name_list = []
price_list = []


for i in range(0,len(res_df)):
  
    name = res_df['상세주소'][i] + ' ' + res_df['사업장명'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))

    time.sleep(3)  
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
    except :
        print('안돼')

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # '리뷰' 탭의 href 속성 가져오기
    review_tab_href = soup.find('a', {'class': 'tpj9w _tab-menu', 'aria-selected': 'true'}).get('href')
    review_url = 'https://pcmap.place.naver.com'+review_tab_href

    driver.get(review_url)
    time.sleep(2.2)  

    for _ in range(4):
    # 스크롤을 위한 JavaScript 코드 실행
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) 

    # HTML 추출
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select('ul.uNsI9 li.nbD78, ul.uNsI9 li.nbD78.Xcpsu')

    # # tag 끝까지 나오게 클릭 
    # try :
    #     while driver.find_elements(By.CLASS_NAME, 'Tvx37'):
    #         try:
    #             button = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[2]/a[1]')
    #             button.click()
    #             time.sleep(0.4)
    #         except ElementClickInterceptedException:
    #             break
    # except NoSuchElementException:
    #     print("No more '더보기' button found. Exiting the loop.")

    # 여러 식당 정보를 반복하며 리스트에 추가
    for item in items:
        restaurant_name_list.append(name)
        category_name_list.append(item.text.split('"')[1])
        price_list.append(int(''.join(filter(str.isdigit, item.text))))

    data_list = {
        '매장명': restaurant_name_list,
        'tag': category_name_list,
        '인원': price_list
    }

result_df = pd.DataFrame(data_list)
result_df

,매장명,tag,인원
0,"강남대로 378, 채선당 샤브보트",음식이 맛있어요,44
1,"강남대로 378, 채선당 샤브보트",재료가 신선해요,30
2,"강남대로 378, 채선당 샤브보트",혼밥하기 좋아요,23
3,"강남대로 378, 채선당 샤브보트",친절해요,14
4,"강남대로 378, 채선당 샤브보트",가성비가 좋아요,9
